In [31]:
# Artificial Neural Network


# Part 1 - Data Preprocessing

# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Importing the dataset
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:13]
y = dataset.iloc[:, 13]

#Create dummy variables
geography=pd.get_dummies(X["Geography"],drop_first=True)
gender=pd.get_dummies(X['Gender'],drop_first=True)

## Concatenate the Data Frames

X=pd.concat([X,geography,gender],axis=1)

## Drop Unnecessary columns
X=X.drop(['Geography','Gender'],axis=1)

# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)



In [32]:
## Perform Hyperparameter Optimization

from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Embedding, Flatten, LeakyReLU, BatchNormalization, Dropout
from tensorflow.keras.activations import relu, sigmoid



def create_model(layers, activation):
    model = Sequential()
    for i, nodes in enumerate(layers):
        if i==0:
            model.add(Dense(nodes,input_dim=X_train.shape[1]))
            model.add(Activation(activation))
            model.add(Dropout(0.3))
        else:
            model.add(Dense(nodes))
            model.add(Activation(activation))
            model.add(Dropout(0.3))
            
    model.add(Dense(units = 1, kernel_initializer= 'glorot_uniform', activation = 'sigmoid')) # Note: no activation beyond this point
    
    model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])
    return model
    
model = KerasClassifier(build_fn=create_model, verbose=0)



In [33]:
layers = ((20), (40, 20), (45, 30, 15))
activations = ('sigmoid', 'relu')
param_grid = dict(layers=layers, activation=activations, batch_size = [128, 256], epochs=[30])
print(param_grid)

{'layers': (20, (40, 20), (45, 30, 15)), 'activation': ('sigmoid', 'relu'), 'batch_size': [128, 256], 'epochs': [30]}


In [34]:
grid = GridSearchCV(estimator=model, param_grid=param_grid,cv=5) #cv= cross validation of 5 

grid_result = grid.fit(X_train, y_train)

[grid_result.best_score_,grid_result.best_params_]


B:\Ana\lib\site-packages\sklearn\model_selection\_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "B:\Ana\lib\site-packages\sklearn\model_selection\_validation.py", line 593, in _fit_and_score
    return_parameters : bool, default=False
  File "B:\Ana\lib\site-packages\tensorflow\python\keras\wrappers\scikit_learn.py", line 223, in fit
    return super(KerasClassifier, self).fit(x, y, **kwargs)
  File "B:\Ana\lib\site-packages\tensorflow\python\keras\wrappers\scikit_learn.py", line 157, in fit
    self.model = self.build_fn(**self.filter_sk_params(self.build_fn))
  File "<ipython-input-32-cad1797b47cf>", line 14, in create_model
    for i, nodes in enumerate(layers):
TypeError: 'int' object is not iterable

  Whether to return the fitted estimator.
B:\Ana\lib\site-packages\sklearn\model_selection\_validation.py:610: FitFailedWarning: Estimator f

[0.8559999942779541,
 {'activation': 'relu',
  'batch_size': 128,
  'epochs': 30,
  'layers': (45, 30, 15)}]

In [35]:
# Puchwa ati ko ki kya galat hora hai?

In [36]:
# Predicting the Test set results
y_pred = grid.predict(X_test)
y_pred = (y_pred > 0.5)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

# Calculate the Accuracy
from sklearn.metrics import accuracy_score
score=accuracy_score(y_pred,y_test)
print(cm)
print('Test accuracy = ',score)

[[1525   70]
 [ 207  198]]
Test accuracy =  0.8615
